In [1]:
import pandas as pd
import googlemaps
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
# get the path of data
path_interim_data = os.environ.get("PATH_INTERIM_DATA_CLEANED")
path_processed_data = os.environ.get("PATH_PROCESSED_DATA")
# get google maps key
gmaps_key = os.environ.get("GOOGLE_MAPS_KEY")

# set filename
filename = 'escolas_municipais.csv'

# set country 
country = 'BR'

# set city name
city = 'São Carlos'

# set name of the place
suffix = 'CEMEI'

In [5]:
# load data
path_data = path_interim_data + filename
data = pd.read_csv(path_data)
data.head()

,Nome,Endereço
0,Amelia Meirelles Botta (Profa),"R.Pericles Soares, 160-Arnon de Mello"
1,Antonio Cotrim (Prof),"R.José Tiberti, 160-Jd.Munique-SC VIII"
2,Bento Prado de Almeida Ferraz Jr (Prof),R.Francisca Dirce Barbosa s/n- Douradinho
3,Bruno Panhoca,"R.Vicente Pelicano, 740-Azulville"
4,Dionisio da Silva,"R.Cristóvão Martinelli, 150 – Sta Eudoxia"


In [6]:
gmaps = googlemaps.Client(key=gmaps_key, queries_per_second=40)

components = {
    'country': country,
    'administrative_area': city
}

In [9]:
no_results = []
more_than_one = []
partial_match = []

for index, row in data.iterrows():
    try:
            
            address = suffix +' '+ row.Nome + ', ' + city
            print(address)
            result = gmaps.geocode(language='pt-BR', address= address, components=components)
            if not result or len(result) > 1 or result[0].get('partial_match') != None:
                result = gmaps.geocode(language='pt-BR', address=row.Endereço + ', ' + city, components=components)
                if not result:
                    print('No results')
                    no_results.append(row)
                    continue
                elif len(result) > 1:
                    print('More than one')
                    more_than_one.append(row)
                    continue
                elif result[0].get('partial_match') != None:
                    print('Partial Match')
                    partial_match.append(row)
                    continue

            data.loc[index, 'LATITUDE'] = result[0]['geometry']['location']['lat']
            data.loc[index, 'LONGITUDE'] = result[0]['geometry']['location']['lng']
    except Exception as e:
        print(e)

CEMEI Amelia Meirelles Botta (Profa), São Carlos
CEMEI Antonio  Cotrim (Prof), São Carlos
CEMEI Bento Prado de Almeida Ferraz Jr (Prof), São Carlos
CEMEI Bruno Panhoca, São Carlos
CEMEI Dionisio da Silva, São Carlos
CEMEI João Muniz, São Carlos
CEMEI João Paulo (papa), São Carlos
CEMEI José Marrara, São Carlos
CEMEI Juliana Maria Ciarrochi Peres, São Carlos
CEMEI Maria Alice Vaz de Macedo (profa), São Carlos
CEMEI Maria Consuelo B.Tolentino, São Carlos
CEMEI Pedro Pucci, São Carlos
CEMEI Ruth Bloem Souto, São Carlos
CEMEI Therezinha Rispoli Massei, São Carlos
CEMEI Homero Frei (prof), São Carlos
CEMEI Maria Lucia Ap Marrara (prof), São Carlos
CEMEI Nilson Ap Gonçalves, São Carlos
CEMEI Olivia Carvalho, São Carlos
CEMEI Paulo Freire (prof), São Carlos
CEMEI Regina Ap. Lima Melchiades, São Carlos
CEMEI Vicente Botta (dep), São Carlos
CEMEI Walter Blanco, São Carlos
CEMEI Antonio de Lourdes Rondon, São Carlos
CEMEI Aracy Leite Pereira Lopes, São Carlos
CEMEI Benedicta Sthal Sodré, São Car

In [10]:
#save file
data.to_csv(path_processed_data+filename, index=False)

,Nome,Endereço,LATITUDE,LONGITUDE
0,Amelia Meirelles Botta (Profa),"R.Pericles Soares, 160-Arnon de Mello",-21.990695,-47.924823
1,Antonio Cotrim (Prof),"R.José Tiberti, 160-Jd.Munique-SC VIII",-21.996285,-47.863849
2,Bento Prado de Almeida Ferraz Jr (Prof),R.Francisca Dirce Barbosa s/n- Douradinho,-22.016116,-47.847449
3,Bruno Panhoca,"R.Vicente Pelicano, 740-Azulville",-22.030831,-47.867076
4,Dionisio da Silva,"R.Cristóvão Martinelli, 150 – Sta Eudoxia",-22.015674,-47.850339
